In [61]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
)
from lightgbm import LGBMClassifier

In [62]:
# Load train/validation/test splits produced in Task 3
train_data = pd.read_csv("data/processed/train.csv")
valid_data = pd.read_csv("data/processed/valid.csv")
test_data  = pd.read_csv("data/processed/test.csv")

# Extract anchored test date
test_date_str = pd.to_datetime(test_data["date_ymd"]).dt.strftime("%Y-%m-%d").unique().tolist()
assert len(test_date_str) == 1, f"Unexpected multiple test dates: {test_date_str}"
test_date_str = test_date_str[0]
test_date = pd.to_datetime(test_date_str)

In [63]:
# We rebuild target-aware splits to prevent label leakage across train/validation/test windows.
 
# Combine splits and rebuild labels
train_data["_src"] = "train"
valid_data["_src"] = "valid"
test_data["_src"]  = "test"
full_data = pd.concat([train_data, valid_data, test_data], ignore_index=True)

# Ensure datetime and sort
full_data["date"] = pd.to_datetime(full_data["date_ymd"], errors="coerce")
full_data = full_data.sort_values(["station_number","date"]).reset_index(drop=True)

# Recompute y_tomorrow if "snow" exists
if "snow" in full_data.columns:
    full_data["snow"] = full_data["snow"].astype(bool)
    full_data["y_tomorrow"] = (
        full_data.groupby("station_number")["snow"].shift(-1).astype("float").fillna(0.0).astype(int)
    )

# Define target date (label reference)
full_data["target_date"] = full_data["date"] + pd.Timedelta(days=1)

# Validation window (60 days before test date)
val_window_days = 60
val_start = test_date - pd.Timedelta(days=val_window_days)

# Splits
train_split = full_data[ full_data["target_date"] <= val_start ].copy()
valid_split = full_data[ (full_data["target_date"] > val_start) & (full_data["target_date"] < test_date) ].copy()
test_split  = full_data[ full_data["date"].dt.strftime("%Y-%m-%d") == test_date_str ].copy()


In [64]:
#We add strict assertions to guarantee that there’s no overlap or leakage across train/valid/test splits.

assert train_split["target_date"].max() <= val_start
if len(valid_split):
    assert valid_split["target_date"].max() < test_date
assert set(train_split.index).isdisjoint(valid_split.index)
assert set(train_split.index).isdisjoint(test_split.index)
assert set(valid_split.index).isdisjoint(test_split.index)

print("Splits validated: no leakage detected.")

Splits validated: no leakage detected.


In [65]:
#We engineer features based only on past information: lags, rolling means, and seasonal encodings. 
#This ensures predictions use only information available up to each day.


base_numeric = ["total_precipitation","mean_temp","max_temperature"]
base_categoric = ["rain","fog","hail","thunder","tornado"]
target_column = "y_tomorrow"

# Cast types
for col in base_numeric:
    full_data[col] = pd.to_numeric(full_data.get(col), errors="coerce")
for col in base_categoric:
    full_data[col] = full_data.get(col, False).astype(bool)

# Sort before lags
full_data = full_data.sort_values(["station_number","date"]).reset_index(drop=True)

# Lags and rolling means
for col in base_numeric:
    full_data[f"{col}_lag1"] = full_data.groupby("station_number")[col].shift(1)
    full_data[f"{col}_lag3"] = full_data.groupby("station_number")[col].shift(3)
    full_data[f"{col}_roll3_mean"] = (
        full_data.groupby("station_number")[col]
            .transform(lambda s: s.rolling(window=3, min_periods=1).mean())
    )

# Seasonality features
full_data["month"] = full_data["date"].dt.month
doy = full_data["date"].dt.dayofyear
full_data["doy_sin"] = np.sin(2*np.pi*doy/365.25)
full_data["doy_cos"] = np.cos(2*np.pi*doy/365.25)

# Snow-specific features
full_data["is_freezing"] = (full_data["mean_temp"] <= 0).astype(int)
full_data["precip_when_freezing"] = full_data["total_precipitation"] * full_data["is_freezing"]
full_data["precip_7d_sum"] = (
    full_data.groupby("station_number")["total_precipitation"].shift(1).rolling(7, min_periods=1).sum()
)


# Final feature set
feature_columns = (
    base_categoric + base_numeric +
    [f"{c}_lag1" for c in base_numeric] +
    [f"{c}_lag3" for c in base_numeric] +
    [f"{c}_roll3_mean" for c in base_numeric] +
    ["is_freezing", "precip_when_freezing", "precip_7d_sum"] +
    ["month","doy_sin","doy_cos"]
)

# Slice back to splits
train_features = full_data.loc[train_split.index, :].copy()
valid_features = full_data.loc[valid_split.index, :].copy()
test_features  = full_data.loc[test_split.index,  :].copy()

# Train/Valid/Test sets
X_train, y_train = train_features[feature_columns], train_features[target_column].astype(int)
X_valid, y_valid = valid_features[feature_columns], valid_features[target_column].astype(int)
X_test,  y_test  = test_features[feature_columns],  test_features[target_column].astype(int)


In [66]:
import seaborn as sns
import matplotlib.pyplot as plt

# Numeric features + target
numeric_features = feature_columns + [target_column]
corr_matrix = full_data[numeric_features].corr()

# Check correlation with target
target_corr = corr_matrix[target_column].sort_values(ascending=False)
print(target_corr)


y_tomorrow                        1.000000
rain                              0.452489
hail                              0.452489
fog                               0.452489
tornado                           0.452489
thunder                           0.452489
total_precipitation               0.051349
doy_cos                           0.045882
total_precipitation_roll3_mean    0.036809
precip_7d_sum                     0.013281
total_precipitation_lag1          0.009586
total_precipitation_lag3         -0.003559
precip_when_freezing             -0.004515
month                            -0.008583
doy_sin                          -0.012363
is_freezing                      -0.013638
mean_temp                        -0.022389
max_temperature                  -0.026504
mean_temp_roll3_mean             -0.038046
mean_temp_lag1                   -0.039824
max_temperature_roll3_mean       -0.051066
max_temperature_lag1             -0.053483
mean_temp_lag3                   -0.053910
max_tempera

In [ ]:

# LightGBM training & tuning

#Why LightGBM is suitable for this task:
#- It efficiently handles tabular data with both numeric (temperatures, precipitation) and categorical (rain, fog, hail) features.  
#- It captures non-linear relationships between weather conditions and snow occurrence.  


# We train a LightGBM classifier. 
# At first, we evaluate the model using a default probability threshold of 0.5.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

# Keep DataFrame column names after imputation
def to_dataframe(X, feature_names):
    return pd.DataFrame(X, columns=feature_names)


model = Pipeline([
    
    ("clf", LGBMClassifier( 
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced"
    ))
])

# Train on training split
model.fit(X_train, y_train)

# Validation predictions
valid_proba = model.predict_proba(X_valid)[:, 1]
valid_pred_default = (valid_proba >= 0.5).astype(int)

print("Validation @ default 0.5 threshold")
print("AUC:", roc_auc_score(y_valid, valid_proba))
print("F1 :", f1_score(y_valid, valid_pred_default))
print("Acc:", accuracy_score(y_valid, valid_pred_default))
print("Prec:", precision_score(y_valid, valid_pred_default))
print("Rec :", recall_score(y_valid, valid_pred_default))
print("ConfusionMatrix:\n", confusion_matrix(y_valid, valid_pred_default))

[LightGBM] [Info] Number of positive: 3514, number of negative: 16500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3246
[LightGBM] [Info] Number of data points in the train set: 20014, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Validation @ default 0.5 threshold
AUC: 0.7417391304347827
F1 : 0.5323741007194245
Acc: 0.7796610169491526
Prec: 0.5
Rec : 0.5692307692307692
ConfusionMatrix:
 [[386  74]
 [ 56  74]]


In [68]:
# Instead of sticking to 0.5, we tune the decision threshold to maximize the F1-score, balancing precision and recall.
# This gives a more robust operational metric.

def tune_threshold(y_true, proba, metric="f1", grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.95, 19)
    best_t, best = 0.5, -1.0
    for t in grid:
        preds = (proba >= t).astype(int)
        score = f1_score(y_true, preds) if metric == "f1" else accuracy_score(y_true, preds)
        if score > best:
            best, best_t = score, t
    return best_t, best

# Tune on validation set
best_threshold, best_f1 = tune_threshold(y_valid, valid_proba, metric="f1")
valid_pred = (valid_proba >= best_threshold).astype(int)

print("Validation @ tuned threshold:", best_threshold)
print("AUC:", roc_auc_score(y_valid, valid_proba))
print("F1 :", f1_score(y_valid, valid_pred))
print("Acc:", accuracy_score(y_valid, valid_pred))
print("Prec:", precision_score(y_valid, valid_pred))
print("Rec :", recall_score(y_valid, valid_pred))
print("ConfusionMatrix:\n", confusion_matrix(y_valid, valid_pred))


Validation @ tuned threshold: 0.5499999999999999
AUC: 0.7417391304347827
F1 : 0.5373134328358209
Acc: 0.7898305084745763
Prec: 0.5217391304347826
Rec : 0.5538461538461539
ConfusionMatrix:
 [[394  66]
 [ 58  72]]


In [69]:
#Once the threshold is tuned, we retrain the model on train+validation data and generate predictions for the test set.


X_train_valid = pd.concat([X_train, X_valid], axis=0)
y_train_valid = pd.concat([y_train, y_valid], axis=0)

model.fit(X_train_valid, y_train_valid)

# Test predictions
test_proba = model.predict_proba(X_test)[:, 1]
test_pred  = (test_proba >= best_threshold).astype(int)

# Save predictions
predictions = test_features[["station_number","date_ymd"]].copy()
predictions["pred_proba_snow_tomorrow"] = test_proba
predictions["pred_label_snow_tomorrow"] = test_pred
Path("data/processed").mkdir(parents=True, exist_ok=True)
predictions.to_csv("data/processed/test_predictions_lightgbm.csv", index=False)

print("Predictions saved to data/processed/test_predictions_lightgbm.csv")

# Test metrics
test_metrics = {
    "AUC": roc_auc_score(y_test, test_proba),
    "F1": f1_score(y_test, test_pred),
    "Accuracy": accuracy_score(y_test, test_pred),
    "Precision": precision_score(y_test, test_pred),
    "Recall": recall_score(y_test, test_pred),
    "ConfusionMatrix": confusion_matrix(y_test, test_pred).tolist(),
    "ThresholdUsed": best_threshold
}

pd.DataFrame([test_metrics]).to_csv("data/processed/test_metrics_lightgbm.csv", index=False)
print("Test metrics saved to data/processed/test_metrics_lightgbm.csv")



[LightGBM] [Info] Number of positive: 3644, number of negative: 16960
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000941 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3252
[LightGBM] [Info] Number of data points in the train set: 20604, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Predictions saved to data/processed/test_predictions_lightgbm.csv
Test metrics saved to data/processed/test_metrics_lightgbm.csv
